In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from skimage import io

local = True
show_images = False

sF_steps = 100
sF_start = 1.5
sF_end = 8

mN_start = 1
mN_end = 10

imgs = {}
img_dir = "images"

if(local):
    img_files = [f for f in os.listdir(img_dir) if not f.startswith(".")]
    imgs = [cv2.imread(os.path.join(img_dir, f)) for f in img_files if not f.startswith(".")]
    imgs = dict(zip(img_files, imgs))
else:
    base_url = "https://github.com/dan91/haarClassifierParameterAnalysis/raw/master/"
    img_url = os.path.join(base_url, "images")
    amount_images = 8
    for i in tqdm(range(1, amount_images + 1)):
        fN = str(i)+".png"
        imgs.update({(fN): io.imread(os.path.join(img_url, fN)) } )


In [ ]:
import numpy as np
import json
import os
import time
from tqdm import tqdm_notebook as tqdm
import urllib.request
if(local):
    with open('labels.json') as f:
        labels = json.load(f)
else:
    labels = json.load(urllib.request.urlopen(os.path.join(base_url, "labels.json")))
    
ids = [d['External ID'] for d in labels if d['Label'] != 'Skip']
coords = [d['Label']['Eye'][0]['geometry'] for d in labels if d['Label'] != 'Skip']
labels = dict(zip(ids,coords))

file_name = "haarcascade_eye.xml"
if(not local):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml", file_name)
cascade = cv2.CascadeClassifier(file_name)


In [ ]:
# https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/visualize.py
def display_images(images, cols):
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image in images:
        plt.subplot(rows, cols, i)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8))
        i += 1
    plt.show()

In [ ]:
%matplotlib inline
count_total = 0
images_after_drawings = []
res = {}
def detect_objects(sF, mN):
    start_time = time.time()
    count_true_pos = count_true_neg = count_false_pos = count_false_neg = 0
    count_image_batch = 1
    for img in imgs:
        current_img = imgs[img].copy()

        (rects, weights, levels) = cascade.detectMultiScale3(imgs[img], scaleFactor=sF, minNeighbors = mN, flags=cv2.CASCADE_SCALE_IMAGE, outputRejectLevels = True
        )
        ground_truth_object = False
        detected_object = False
        detected_object_correct_position = False

        # this is needed to show a green/red cirlce in upper left corner if there is no ground truth object 
        x_t = y_t = 5
        # image contains object
        if(img in labels):
            ground_truth_object = True
            x_t = labels[img]['x']
            y_t = labels[img]['y']

        # if openCV doesn't detect any objects, rects is a tuple (not a np array) and we skip it   
        if(type(rects) is np.ndarray):
            detected_object = True
            # we only look at the object with the highest weight         
            heighest_weight = (weights.tolist().index(max(weights)))
            rect = rects[heighest_weight]
            x1, y1, x2, y2 = rect
            cv2.rectangle(current_img, (x1, y1), (x2, y2), (255,255,255,255), 5)

            if(ground_truth_object):
                # check if ground truth object is within detected object 
                if((x_t > x1 and x_t < x2) and (y_t > y1 and y_t < y2)):
                    detected_object_correct_position = True

        # here we determine the outcome of the prediction and print a colored ground truth circle 
        # (or a circle in the upper left corner if no ground truth object present)
        # (green = true, red = false).
        if(ground_truth_object and detected_object_correct_position):
            count_true_pos += 1
            c = (0, 255, 0, 255)
        elif(ground_truth_object and not detected_object):
            count_false_neg += 1
            c = (255, 0, 0, 255)
        elif(ground_truth_object and not detected_object_correct_position):
            count_false_pos += 1
            c = (255, 0, 0, 255)
        elif(not ground_truth_object and detected_object):
            count_false_pos += 1
            c = (255, 0, 0, 255)
        elif(not ground_truth_object and not detected_object):
            count_true_neg += 1
            c = (0, 255, 0, 255)

        cv2.circle(current_img, (x_t, y_t), 3, c, 20)
        images_after_drawings.append(current_img)

    res.update({(sF,mN): { 
        "duration": (time.time() - start_time)/len(imgs), 
        "f1": (2*count_true_pos)/(2*count_true_pos+count_false_pos+count_false_neg)}}
    )
    if(show_images):
        print(" --- scaleFactor: {}, minNeightbors: {}".format(sF, mN))
        display_images(images_after_drawings[-8:], cols=len(imgs))

In [ ]:
for sF in tqdm(np.linspace(sF_start, sF_end, sF_steps)):
    for mN in range(mN_start, mN_end):
        detect_objects(sF, mN)



In [ ]:
# https://stackoverflow.com/a/47230966
def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

configure_plotly_browser_state()

init_notebook_mode(connected=False)

t = [v['duration'] for v in res.values()]
a = [v['f1'] for v in res.values()]
text = [str(k) for k in res.keys()]
# Create a trace
trace = go.Scatter(
    x = t,
    y = a,
    mode = 'markers',
    text = text,
)

data = [trace]
layout = go.Layout(
    yaxis=dict(
        range=[0, 1]
    )
)

       
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-line.html')